In [1]:
import requests        
import json            
import pandas as pd    
import numpy as np     

import datetime as dt  

import os # for api keys

from binance.client import Client # import binance client



In [2]:
# bring in api keys from system

api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')

In [3]:
# initiate client with apis

client = Client(api_key, api_secret)

In [4]:
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor

In [5]:
# pull data in from Binance API

btc_hist = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "1 Jul, 2019")

In [6]:
print(btc_hist)

[[1561939200000, '10854.10000000', '11282.28000000', '10030.00000000', '10624.93000000', '90962.26827100', 1562025599999, '972343281.86035535', 757458, '44210.98836500', '473020897.41183619', '0'], [1562025600000, '10624.90000000', '10938.75000000', '9727.00000000', '10842.85000000', '109561.03872800', 1562111999999, '1128353804.80079160', 870148, '54998.50360600', '566863180.55092289', '0'], [1562112000000, '10844.98000000', '11991.89000000', '10841.04000000', '11940.00000000', '96815.90029000', 1562198399999, '1097804642.75777372', 814754, '49613.61674400', '561585673.86519116', '0'], [1562198400000, '11940.00000000', '12000.00000000', '11055.00000000', '11145.67000000', '66512.22189200', 1562284799999, '775818897.38948483', 643867, '33626.85186200', '392411806.43340068', '0'], [1562284800000, '11145.67000000', '11406.83000000', '10796.44000000', '10970.73000000', '63534.35058200', 1562371199999, '705853649.02648962', 564857, '32306.91425900', '359142813.98810695', '0'], [15623712000

In [7]:
type(btc_hist)

list

In [8]:
# create dataframe from Binance list data

btc_df = pd.DataFrame(btc_hist, columns=['openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numTrade', 'takerBuyBaseAssetVolume', 'takerByQuoteAssetVolume', 'ignore'])

In [9]:
# drop columns that are of no use

btc_df.drop(['openTime', 'quoteAssetVolume', 'numTrade', 'takerBuyBaseAssetVolume', 'takerByQuoteAssetVolume', 'ignore'], axis=1, inplace=True)

In [10]:
# rename columns

btc_df = btc_df[['closeTime', 'open', 'high', 'low', 'close', 'volume']]

In [11]:
# convert "closeTime" to readable date from miliseconds

btc_df['closeTime'] = pd.to_datetime(btc_df['closeTime'], unit='ms')

In [12]:
# drop time sequence from date in "closeTime"

btc_df['closeTime'] = pd.to_datetime(btc_df['closeTime']).dt.date

In [13]:
# set index to "closeTime"

btc_df.set_index(['closeTime'], inplace=True)

In [14]:
# rename index "closeTime" to "Date"

btc_df.index.names = ['Date']

In [15]:
# convert columns from str to float (for indicators)

btc_df[['open','high', 'low', 'close', 'volume']]=btc_df[['open','high', 'low', 'close', 'volume']].astype(float)

In [16]:
btc_df.head()

,open,high,low,close,volume
Date,,,,,
2019-07-01,10854.10,11282.28,10030.00,10624.93,90962.268271
2019-07-02,10624.90,10938.75,9727.00,10842.85,109561.038728
2019-07-03,10844.98,11991.89,10841.04,11940.00,96815.900290
2019-07-04,11940.00,12000.00,11055.00,11145.67,66512.221892
2019-07-05,11145.67,11406.83,10796.44,10970.73,63534.350582


In [17]:
# import technical indicator library 

import btalib

In [18]:
btc_df.head()

,open,high,low,close,volume
Date,,,,,
2019-07-01,10854.10,11282.28,10030.00,10624.93,90962.268271
2019-07-02,10624.90,10938.75,9727.00,10842.85,109561.038728
2019-07-03,10844.98,11991.89,10841.04,11940.00,96815.900290
2019-07-04,11940.00,12000.00,11055.00,11145.67,66512.221892
2019-07-05,11145.67,11406.83,10796.44,10970.73,63534.350582


In [19]:
# create RSI indicator

rsi = btalib.rsi(btc_df, period=14)

In [20]:
# join RSI to dataframe

btc_df = btc_df.join([rsi.df])

In [21]:
# create EMA long version and rename to "ema_long"

ema_long = btalib.ema(btc_df.close, period=50)
ema_long.df.rename(columns={'ema': 'ema_long'}, inplace=True)

In [22]:
# create EMA short version and rename to "ema_short"

ema_short = btalib.ema(btc_df.close, period=20)
ema_short.df.rename(columns={'ema': 'ema_short'}, inplace=True)

In [23]:
# join ema_long and ema_short to dataframe

btc_df = btc_df.join([ema_long.df, ema_short.df])

In [24]:
btc_df.head()

,open,high,low,close,volume,rsi,ema_long,ema_short
Date,,,,,,,,
2019-07-01,10854.10,11282.28,10030.00,10624.93,90962.268271,NaN,NaN,NaN
2019-07-02,10624.90,10938.75,9727.00,10842.85,109561.038728,NaN,NaN,NaN
2019-07-03,10844.98,11991.89,10841.04,11940.00,96815.900290,NaN,NaN,NaN
2019-07-04,11940.00,12000.00,11055.00,11145.67,66512.221892,NaN,NaN,NaN
2019-07-05,11145.67,11406.83,10796.44,10970.73,63534.350582,NaN,NaN,NaN


In [25]:
# create ATR indicator

atr = btalib.atr(btc_df, period=14)

In [26]:
# create ADX indicator

adx = btalib.adx(btc_df, period=14)

In [27]:
# join ATR and ADX to dataframe

btc_df = btc_df.join([atr.df, adx.df])

In [28]:
# create OBV indicator

obv = btalib.obv(btc_df)

In [29]:
# join OBV to indicator

btc_df = btc_df.join([obv.df])

In [30]:
# create "prediction target" column (shift new column values up by 1)

btc_df["close_nextday"]=btc_df.close.shift(-1)

In [31]:
btc_df.head(60)

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday
Date,,,,,,,,,,,,
2019-07-01,10854.10,11282.28,10030.00,10624.93,90962.268271,NaN,NaN,NaN,NaN,NaN,NaN,10842.85
2019-07-02,10624.90,10938.75,9727.00,10842.85,109561.038728,NaN,NaN,NaN,NaN,NaN,109561.038728,11940.00
2019-07-03,10844.98,11991.89,10841.04,11940.00,96815.900290,NaN,NaN,NaN,NaN,NaN,206376.939018,11145.67
2019-07-04,11940.00,12000.00,11055.00,11145.67,66512.221892,NaN,NaN,NaN,NaN,NaN,139864.717126,10970.73
2019-07-05,11145.67,11406.83,10796.44,10970.73,63534.350582,NaN,NaN,NaN,NaN,NaN,76330.366544,11256.49
2019-07-06,10982.41,11665.00,10964.51,11256.49,51469.496331,NaN,NaN,NaN,NaN,NaN,127799.862875,11406.24
2019-07-07,11256.45,11538.00,11094.37,11406.24,38884.795599,NaN,NaN,NaN,NaN,NaN,166684.658474,12238.60
2019-07-08,11410.00,12338.03,11220.00,12238.60,52182.367215,NaN,NaN,NaN,NaN,NaN,218867.025689,12543.41
2019-07-09,12238.60,12794.73,12068.00,12543.41,78442.130343,NaN,NaN,NaN,NaN,NaN,297309.156032,12108.37


In [32]:
# drop unecessary rows not in project time range (also removes NaN values created by indicators)

btc_df.drop(btc_df.head(63).index,inplace=True)

In [33]:
btc_df.head()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday
Date,,,,,,,,,,,,
2019-09-02,9723.59,10450.00,9712.50,10340.00,44740.248093,51.381478,10452.265343,10164.518939,530.693553,19.114750,225053.863244,10615.28
2019-09-03,10340.00,10773.00,10272.00,10615.28,47998.376781,54.904081,10458.658074,10207.448563,528.572585,18.426961,273052.240025,10567.02
2019-09-04,10611.85,10799.00,10369.89,10567.02,43943.889026,54.163174,10462.907561,10241.693462,521.468114,17.842351,229108.350999,10564.49
2019-09-05,10565.92,10900.00,10450.00,10564.49,33970.960639,54.121941,10466.891187,10272.435990,516.363249,17.516155,195137.390360,10298.73
2019-09-06,10563.13,10905.87,10150.00,10298.73,58799.640959,49.830575,10460.296630,10274.940181,533.470874,16.423958,136337.749401,10455.88


In [34]:
# bring BTC futures prices

btc_fut = pd.read_csv("closeFUT.csv", index_col='Date', parse_dates=True)

In [35]:
btc_fut.shape

(443, 1)

In [36]:
btc_df.shape

(443, 12)

In [37]:
# join btc_fut dataframe to original btc_df

btc_df = btc_df.join(btc_fut, how="inner")

In [38]:
btc_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 443 entries, 2019-09-02 to 2020-11-17
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   open           443 non-null    float64
 1   high           443 non-null    float64
 2   low            443 non-null    float64
 3   close          443 non-null    float64
 4   volume         443 non-null    float64
 5   rsi            443 non-null    float64
 6   ema_long       443 non-null    float64
 7   ema_short      443 non-null    float64
 8   atr            443 non-null    float64
 9   adx            443 non-null    float64
 10  obv            443 non-null    float64
 11  close_nextday  442 non-null    float64
 12  closeFUT       443 non-null    float64
dtypes: float64(13)
memory usage: 68.5 KB


In [39]:
btc_df.head()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday,closeFUT
Date,,,,,,,,,,,,,
2019-09-02,9723.59,10450.00,9712.50,10340.00,44740.248093,51.381478,10452.265343,10164.518939,530.693553,19.114750,225053.863244,10615.28,10341.07
2019-09-03,10340.00,10773.00,10272.00,10615.28,47998.376781,54.904081,10458.658074,10207.448563,528.572585,18.426961,273052.240025,10567.02,10610.72
2019-09-04,10611.85,10799.00,10369.89,10567.02,43943.889026,54.163174,10462.907561,10241.693462,521.468114,17.842351,229108.350999,10564.49,10568.07
2019-09-05,10565.92,10900.00,10450.00,10564.49,33970.960639,54.121941,10466.891187,10272.435990,516.363249,17.516155,195137.390360,10298.73,10565.04
2019-09-06,10563.13,10905.87,10150.00,10298.73,58799.640959,49.830575,10460.296630,10274.940181,533.470874,16.423958,136337.749401,10455.88,10294.96


In [40]:
# round volume column

btc_df['volume'] = btc_df['volume'].round(2)

In [41]:
btc_df.head()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday,closeFUT
Date,,,,,,,,,,,,,
2019-09-02,9723.59,10450.00,9712.50,10340.00,44740.25,51.381478,10452.265343,10164.518939,530.693553,19.114750,225053.863244,10615.28,10341.07
2019-09-03,10340.00,10773.00,10272.00,10615.28,47998.38,54.904081,10458.658074,10207.448563,528.572585,18.426961,273052.240025,10567.02,10610.72
2019-09-04,10611.85,10799.00,10369.89,10567.02,43943.89,54.163174,10462.907561,10241.693462,521.468114,17.842351,229108.350999,10564.49,10568.07
2019-09-05,10565.92,10900.00,10450.00,10564.49,33970.96,54.121941,10466.891187,10272.435990,516.363249,17.516155,195137.390360,10298.73,10565.04
2019-09-06,10563.13,10905.87,10150.00,10298.73,58799.64,49.830575,10460.296630,10274.940181,533.470874,16.423958,136337.749401,10455.88,10294.96


In [42]:
# drop 2 tail rows to fit tweet sentiment dataframe and match time range

btc_df.drop(btc_df.tail(2).index,inplace=True)

In [43]:
btc_df.tail()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday,closeFUT
Date,,,,,,,,,,,,,
2020-11-11,15297.21,15965.00,15272.68,15684.24,78469.75,73.450202,12927.689795,14236.176564,698.560946,53.701439,1.954724e+06,16291.86,15685.02
2020-11-12,15684.25,16340.70,15440.64,16291.86,102196.36,76.977504,13059.618039,14431.955939,712.953736,53.845549,2.056921e+06,16320.70,16289.98
2020-11-13,16291.85,16480.00,15952.35,16320.70,75691.88,77.132794,13187.503606,14611.836326,699.717755,54.091743,2.132613e+06,16070.45,16321.89
2020-11-14,16320.04,16326.99,15670.00,16070.45,59116.35,72.559314,13300.560327,14750.751914,696.665772,53.511714,2.073496e+06,15957.00,16066.20
2020-11-15,16069.56,16180.00,15774.72,15957.00,43596.84,70.517941,13404.734432,14865.632684,675.852503,52.973116,2.029899e+06,16713.57,15952.73


In [44]:
# bring in tweet sentiment dataframe

tweet_sentiment_df = pd.read_csv('btc_sentiment.csv', index_col='date', parse_dates=True)

In [45]:
tweet_sentiment_df.head()

,tweet_sentiment
date,
2019-09-02,-1.0
2019-09-03,0.5
2019-09-04,0.5
2019-09-05,0.5
2019-09-06,0.0


In [46]:
# join tweet sentiment dataframe with btc_df

btc_df = btc_df.join(tweet_sentiment_df, how="inner")

In [47]:
btc_df.head()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday,closeFUT,tweet_sentiment
Date,,,,,,,,,,,,,,
2019-09-02,9723.59,10450.00,9712.50,10340.00,44740.25,51.381478,10452.265343,10164.518939,530.693553,19.114750,225053.863244,10615.28,10341.07,-1.0
2019-09-03,10340.00,10773.00,10272.00,10615.28,47998.38,54.904081,10458.658074,10207.448563,528.572585,18.426961,273052.240025,10567.02,10610.72,0.5
2019-09-04,10611.85,10799.00,10369.89,10567.02,43943.89,54.163174,10462.907561,10241.693462,521.468114,17.842351,229108.350999,10564.49,10568.07,0.5
2019-09-05,10565.92,10900.00,10450.00,10564.49,33970.96,54.121941,10466.891187,10272.435990,516.363249,17.516155,195137.390360,10298.73,10565.04,0.5
2019-09-06,10563.13,10905.87,10150.00,10298.73,58799.64,49.830575,10460.296630,10274.940181,533.470874,16.423958,136337.749401,10455.88,10294.96,0.0


In [48]:
# reorder columns

btc_df = btc_df[['open', 'high', 'low', 'close', 'closeFUT', 'volume', 'rsi', 'ema_short', 'ema_long', 'atr', 'adx', 'obv', 'tweet_sentiment', 'close_nextday']]

In [49]:
btc_df.head()

,open,high,low,close,closeFUT,volume,rsi,ema_short,ema_long,atr,adx,obv,tweet_sentiment,close_nextday
Date,,,,,,,,,,,,,,
2019-09-02,9723.59,10450.00,9712.50,10340.00,10341.07,44740.25,51.381478,10164.518939,10452.265343,530.693553,19.114750,225053.863244,-1.0,10615.28
2019-09-03,10340.00,10773.00,10272.00,10615.28,10610.72,47998.38,54.904081,10207.448563,10458.658074,528.572585,18.426961,273052.240025,0.5,10567.02
2019-09-04,10611.85,10799.00,10369.89,10567.02,10568.07,43943.89,54.163174,10241.693462,10462.907561,521.468114,17.842351,229108.350999,0.5,10564.49
2019-09-05,10565.92,10900.00,10450.00,10564.49,10565.04,33970.96,54.121941,10272.435990,10466.891187,516.363249,17.516155,195137.390360,0.5,10298.73
2019-09-06,10563.13,10905.87,10150.00,10298.73,10294.96,58799.64,49.830575,10274.940181,10460.296630,533.470874,16.423958,136337.749401,0.0,10455.88


In [105]:
# check for any missing columns

btc_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 441 entries, 2019-09-02 to 2020-11-15
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   open             441 non-null    float64
 1   high             441 non-null    float64
 2   low              441 non-null    float64
 3   close            441 non-null    float64
 4   closeFUT         441 non-null    float64
 5   volume           441 non-null    float64
 6   rsi              441 non-null    float64
 7   ema_short        441 non-null    float64
 8   ema_long         441 non-null    float64
 9   atr              441 non-null    float64
 10  adx              441 non-null    float64
 11  obv              441 non-null    float64
 12  tweet_sentiment  441 non-null    float64
 13  close_nextday    441 non-null    float64
dtypes: float64(14)
memory usage: 71.7 KB
